In [1]:
import pandas as pd
import numpy as np
from DS_Planck_Unet import val_pix, east_val, west_val
from DS_models_stat import stat_orig_cats, stat_split_cats, do_all_stats, stat_cat, change_df
import seaborn as sns
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from DS_data_transformation import inter_cats

In [2]:
match_dist=600/3600

In [3]:
format_fun = lambda x: '{:.3f}'.format(x) if not (float(x).is_integer() or type(x) == type('')) else x
prm = 'N_cl - N_PSZ2'
files3 = { 
        'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8.csv',
        'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8.csv',
        'SZcat(y-map)' : '/home/rt2122/Data/SRGz/nilc_src_0.3deg_mar12_matched.csv',
        'PSZ2' : '/home/rt2122/Data/original_catalogs/csv/PSZ2.csv'
        }
names = ['SZcat(y-map)', 'SZcat(base)', 'SZcat(AL)', 'SZcat(gen)', 'PSZ2']

vals_default = {'SZcat(AL)' : 0.0, 'SZcat(base)' : 0.0, 'SZcat(y-map)' : -np.inf, 'SZcat(gen)' : '0.00', 'PSZ2' : ''}
vals_rel = {'SZcat(AL)' : 0.775, 'SZcat(base)' : 0.905, 'SZcat(y-map)' : 10.5, 'SZcat(gen)' : '0.81', 'PSZ2' : ''}
prm_lines = ['s/n >= {:.2f}', 'p_max >= {:.2f}', 'p_max >= {:.2f}', 'mpp_er >= {}', 'psz2.{}']

In [4]:
def calc_line(name, dict_cut, big_pix, prm_line, e_mode=False):
    recall = change_df(stat_cat({name : files3[name]}, big_pix=big_pix, dict_cut=dict_cut, 
                  match_dist=match_dist, spec_precision=['eROSITA', 'PSZ2_inter_all_true', 
                                                         'PSZ2_inter_eROSITA', 'PSZ2', 'all_true'], use_err=True), 
                       format_s=format_fun)
    
    recall[prm] = recall['found_all_true'] - recall['found_PSZ2_inter_all_true']
    if e_mode:
        recall[prm] = recall['found_eROSITA'] - recall['found_PSZ2_inter_eROSITA']
        
    recall['prm'] = prm_line 
    return recall

In [21]:
#таблица 3 первая половина
vals = vals_default
files3['SZcat(gen)'] = '/home/rt2122/Data/gen_cats_er/er_gen_all_found_brcat_prec{}.csv'.format(vals['SZcat(gen)'])

df1 = []
big_pix = [41]
l_val = [-np.inf, np.inf]
b_val = [-np.inf, np.inf]
prm_list = ['PSZ2(z)', 'err_PSZ2(z)', 'MCXC', 'err_MCXC', 'ACT', 'err_ACT', 'all_true', 
            'precision', prm, 'all', 'prm']
prm_list1 = ['PSZ2(z)', 'MCXC', 'ACT', 'all_true', 'precision', prm, 'all', 'found_PSZ2']

dict_cut_list = [{'s/n' : [vals[names[0]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[1]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[2]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                {'l' : l_val, 'b' : b_val}]

for name, dict_cut, prm_line in zip(names, dict_cut_list, prm_lines):
    df1.append(calc_line(name=name, dict_cut=dict_cut, big_pix=big_pix, prm_line=prm_line.format(vals[name])))
df1 = pd.concat(df1)

In [24]:
df1 #3 

,ACT,PSZ2,precision_PSZ2,found_PSZ2,err_PSZ2,MCXC,PSZ2(z),all_true,precision_all_true,found_all_true,err_all_true,PSZ2_inter_all_true,precision_PSZ2_inter_all_true,found_PSZ2_inter_all_true,err_PSZ2_inter_all_true,found,precision,all,N_cl - N_PSZ2,prm
SZcat(y-map),0.262,0.914,0.104,55,2,0.471,0.925,0.282,0.223,118,16,0.927,0.100,53,2,120,0.226,530,65,s/n >= -inf
SZcat(base),0.259,0.914,0.151,55,2,0.515,0.925,0.290,0.321,117,14,0.927,0.146,53,2,119,0.327,364,64,p_max >= 0.00
SZcat(AL),0.284,0.897,0.126,54,2,0.515,0.906,0.311,0.295,127,15,0.909,0.121,52,2,129,0.300,430,75,p_max >= 0.00
SZcat(gen),0.320,0.879,0.071,54,3,0.515,0.887,0.339,0.192,147,22,0.891,0.068,52,3,149,0.195,764,95,mpp_er >= 0.00
PSZ2,0.116,1.0,1.0,58,0,0.397,1.0,0.149,0.948,55,3,1.0,0.948,55,0,58,1.0,58,0,psz2.


In [25]:
print(df1[prm_list1].to_latex())

\begin{tabular}{llllllrrr}
\toprule
{} & PSZ2(z) &   MCXC &    ACT & all\_true & precision &  N\_cl - N\_PSZ2 &  all &  found\_PSZ2 \\
\midrule
SZcat(y-map) &   0.925 &  0.471 &  0.262 &    0.282 &     0.226 &             65 &  530 &          55 \\
SZcat(base)  &   0.925 &  0.515 &  0.259 &    0.290 &     0.327 &             64 &  364 &          55 \\
SZcat(AL)    &   0.906 &  0.515 &  0.284 &    0.311 &     0.300 &             75 &  430 &          54 \\
SZcat(gen)   &   0.887 &  0.515 &  0.320 &    0.339 &     0.195 &             95 &  764 &          54 \\
PSZ2         &     1.0 &  0.397 &  0.116 &    0.149 &       1.0 &              0 &   58 &          58 \\
\bottomrule
\end{tabular}



In [26]:
#таблица 3 вторая половина
vals = vals_rel
files3['SZcat(gen)'] = '/home/rt2122/Data/gen_cats_er/er_gen_all_found_brcat_prec{}.csv'.format(vals['SZcat(gen)'])

df2 = []

dict_cut_list = [{'s/n' : [vals[names[0]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[1]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[2]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                ]

for name, dict_cut, prm_line in zip(names, dict_cut_list, prm_lines):
    df2.append(calc_line(name=name, dict_cut=dict_cut, big_pix=big_pix, prm_line=prm_line.format(vals[name])))
df2 = pd.concat(df2)

In [27]:
df2 #3 prec0.8

,ACT,PSZ2,precision_PSZ2,found_PSZ2,err_PSZ2,MCXC,PSZ2(z),all_true,precision_all_true,found_all_true,err_all_true,PSZ2_inter_all_true,precision_PSZ2_inter_all_true,found_PSZ2_inter_all_true,err_PSZ2_inter_all_true,found,precision,all,N_cl - N_PSZ2,prm
SZcat(y-map),0.070,0.448,1.0,26,0,0.221,0.491,0.065,1.0,26,2,0.473,1.0,26,0,26,1.0,26,0,s/n >= 10.50
SZcat(base),0.070,0.552,0.780,32,0,0.324,0.585,0.091,0.829,34,2,0.582,0.780,32,0,34,0.829,41,2,p_max >= 0.91
SZcat(AL),0.091,0.672,0.796,39,0,0.353,0.717,0.115,0.878,43,2,0.709,0.796,39,0,43,0.878,49,4,p_max >= 0.78
SZcat(gen),0.091,0.672,0.796,39,0,0.353,0.717,0.115,0.878,43,2,0.709,0.796,39,0,43,0.878,49,4,mpp_er >= 0.81
PSZ2,0.116,1.0,1.0,58,0,0.397,1.0,0.149,0.948,55,3,1.0,0.948,55,0,58,1.0,58,0,psz2.


In [28]:
print(df2[prm_list1].to_latex())

\begin{tabular}{llllllrrr}
\toprule
{} & PSZ2(z) &   MCXC &    ACT & all\_true & precision &  N\_cl - N\_PSZ2 &  all &  found\_PSZ2 \\
\midrule
SZcat(y-map) &   0.491 &  0.221 &  0.070 &    0.065 &       1.0 &              0 &   26 &          26 \\
SZcat(base)  &   0.585 &  0.324 &  0.070 &    0.091 &     0.829 &              2 &   41 &          32 \\
SZcat(AL)    &   0.717 &  0.353 &  0.091 &    0.115 &     0.878 &              4 &   49 &          39 \\
SZcat(gen)   &   0.717 &  0.353 &  0.091 &    0.115 &     0.878 &              4 &   49 &          39 \\
PSZ2         &     1.0 &  0.397 &  0.116 &    0.149 &       1.0 &              0 &   58 &          58 \\
\bottomrule
\end{tabular}



In [29]:
#таблица 4 первая половина
vals = vals_default
files3['SZcat(gen)'] = '/home/rt2122/Data/gen_cats_er/er_gen_all_found_brcat_prec{}.csv'.format(vals['SZcat(gen)'])

df3 = []
big_pix = list(range(48))
l_val = [0, 180]
b_val = [20, np.inf]
prm_list = ['PSZ2(z)', 'err_PSZ2(z)', 'MCXC', 'err_MCXC', 'ACT', 'eROSITA', 'precision_eROSITA', prm, 'all', 'prm']
prm_list1 = ['PSZ2(z)', 'MCXC', 'ACT', 'eROSITA', 'precision_eROSITA', prm, 'all', 'found_PSZ2']

dict_cut_list = [{'s/n' : [vals[names[0]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[1]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[2]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                ]

for name, dict_cut, prm_line in zip(names, dict_cut_list, prm_lines):
    df3.append(calc_line(name=name, dict_cut=dict_cut, big_pix=big_pix, prm_line=prm_line.format(vals[name]), e_mode=True))
df3 = pd.concat(df3)

In [30]:
df3 #4

,ACT,PSZ2,precision_PSZ2,found_PSZ2,err_PSZ2,MCXC,eROSITA,precision_eROSITA,found_eROSITA,err_eROSITA,...,err_PSZ2_inter_all_true,PSZ2_inter_eROSITA,precision_PSZ2_inter_eROSITA,found_PSZ2_inter_eROSITA,err_PSZ2_inter_eROSITA,found,precision,all,N_cl - N_PSZ2,prm
SZcat(y-map),0.202,0.893,0.094,647,30,0.413,0.085,0.316,2166,934,...,21,0.940,0.080,547,19,2308,0.336,6862,1619,s/n >= -inf
SZcat(base),0.247,0.928,0.087,664,25,0.432,0.093,0.308,2348,926,...,15,0.961,0.073,556,19,2517,0.330,7618,1792,p_max >= 0.00
SZcat(AL),0.253,0.938,0.081,670,24,0.421,0.098,0.305,2515,1014,...,14,0.969,0.068,558,16,2679,0.324,8259,1957,p_max >= 0.00
SZcat(gen),0.271,0.931,0.057,682,40,0.458,0.117,0.272,3283,1513,...,26,0.961,0.047,568,26,3492,0.290,12058,2715,mpp_er >= 0.00
PSZ2,0.099,0.993,1.0,692,5,0.304,0.033,0.837,579,110,...,4,0.996,0.812,562,3,692,1.0,692,17,psz2.


In [31]:
print(df3[prm_list1].to_latex())

\begin{tabular}{llllllrrr}
\toprule
{} & PSZ2(z) &   MCXC &    ACT & eROSITA & precision\_eROSITA &  N\_cl - N\_PSZ2 &    all &  found\_PSZ2 \\
\midrule
SZcat(y-map) &   0.926 &  0.413 &  0.202 &   0.085 &             0.316 &           1619 &   6862 &         647 \\
SZcat(base)  &   0.963 &  0.432 &  0.247 &   0.093 &             0.308 &           1792 &   7618 &         664 \\
SZcat(AL)    &   0.967 &  0.421 &  0.253 &   0.098 &             0.305 &           1957 &   8259 &         670 \\
SZcat(gen)   &   0.951 &  0.458 &  0.271 &   0.117 &             0.272 &           2715 &  12058 &         682 \\
PSZ2         &   0.994 &  0.304 &  0.099 &   0.033 &             0.837 &             17 &    692 &         692 \\
\bottomrule
\end{tabular}



In [32]:
#таблица 4 вторая половина
vals = vals_rel
files3['SZcat(gen)'] = '/home/rt2122/Data/gen_cats_er/er_gen_all_found_brcat_prec{}.csv'.format(vals['SZcat(gen)'])

df4 = []

dict_cut_list = [{'s/n' : [vals[names[0]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[1]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[2]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                ]

for name, dict_cut, prm_line in zip(names, dict_cut_list, prm_lines):
    df4.append(calc_line(name=name, dict_cut=dict_cut, big_pix=big_pix, prm_line=prm_line.format(vals[name]), e_mode=True))
df4 = pd.concat(df4)

In [33]:
df4 #4 prec0.8

,ACT,PSZ2,precision_PSZ2,found_PSZ2,err_PSZ2,MCXC,eROSITA,precision_eROSITA,found_eROSITA,err_eROSITA,...,err_PSZ2_inter_all_true,PSZ2_inter_eROSITA,precision_PSZ2_inter_eROSITA,found_PSZ2_inter_eROSITA,err_PSZ2_inter_eROSITA,found,precision,all,N_cl - N_PSZ2,prm
SZcat(y-map),0.043,0.301,0.741,209,1,0.160,0.014,0.819,231,52,...,1,0.385,0.723,204,1,236,0.837,282,27,s/n >= 10.50
SZcat(base),0.094,0.601,0.705,415,0,0.252,0.027,0.822,484,80,...,0,0.694,0.672,396,1,501,0.851,589,88,p_max >= 0.91
SZcat(AL),0.111,0.697,0.691,482,2,0.303,0.033,0.837,584,105,...,2,0.799,0.646,451,3,610,0.874,698,133,p_max >= 0.78
SZcat(gen),0.111,0.697,0.692,482,2,0.303,0.033,0.838,584,105,...,2,0.799,0.648,452,3,610,0.875,697,132,mpp_er >= 0.81
PSZ2,0.099,0.993,1.0,692,5,0.304,0.033,0.837,579,110,...,4,0.996,0.812,562,3,692,1.0,692,17,psz2.


In [34]:
print(df4[prm_list1].to_latex())

\begin{tabular}{llllllrrr}
\toprule
{} & PSZ2(z) &   MCXC &    ACT & eROSITA & precision\_eROSITA &  N\_cl - N\_PSZ2 &  all &  found\_PSZ2 \\
\midrule
SZcat(y-map) &   0.389 &  0.160 &  0.043 &   0.014 &             0.819 &             27 &  282 &         209 \\
SZcat(base)  &   0.732 &  0.252 &  0.094 &   0.027 &             0.822 &             88 &  589 &         415 \\
SZcat(AL)    &   0.816 &  0.303 &  0.111 &   0.033 &             0.837 &            133 &  698 &         482 \\
SZcat(gen)   &   0.816 &  0.303 &  0.111 &   0.033 &             0.838 &            132 &  697 &         482 \\
PSZ2         &   0.994 &  0.304 &  0.099 &   0.033 &             0.837 &             17 &  692 &         692 \\
\bottomrule
\end{tabular}

